# Read HDF files from results database
This notebook shows how to read, visualize and process openmc results. In addition, it shows how to store them in an HDF file for the `\results_database\` folder of a given benchmark. To do that we will take advantage of the `\example results\` and `\example_database\` folders. In the former we stored an example openmc `statepoint` file (`example_statepoint.100.h5`) to read. In the latter we will store our new `my_openmc_results.h5` file with the results rearranged consistently with the other HDF files already present in the `\results_database\` folders.

In [1]:
import openmc_fusion_benchmarks as ofb
import numpy as np

Read openmc `statepoint` file:

In [2]:
openmc_results = ofb.ResultsFromOpenmc(statepoint_file='mystatepoint.100.h5', path='results')

Unfortunately we need a bit of postoprocessing before storing them in a new HDF file. Most likely we need to divide the tally by the geometry filter dimensions (e.g. surface area, cell volume, mesh element volume etc.) but other steps might be necessary, depending on the tally. Here we have the volume of the cells.

In [3]:
openmc_results.list_tallies()

vessel_neutron_spectra_tally
vessel_photon_spectra_tally


In [4]:
# _foil_volume = .1 * 1.8**2/4 * np.pi
# volumes_onaxis1 = np.concatenate((np.ones(5), np.ones(4)*2, np.ones(4)*3)) * _foil_volume

df = openmc_results.get_tally_dataframe(tally_name='vessel_neutron_spectra_tally')
del df['surface']
del df['particle']
del df['nuclide']
del df['score']

df

,energy low [eV],energy high [eV],mean,std. dev.
0,86140.0,89650.0,0.000864,1.111672e-07
1,89650.0,93310.0,0.000967,9.781908e-08
2,93310.0,97120.0,0.000967,1.123191e-07
3,97120.0,101090.0,0.000972,1.183270e-07
4,101090.0,105210.0,0.001070,1.169516e-07
...,...,...,...,...
132,16915000.0,17605000.0,0.000113,3.088201e-08
133,17605000.0,18324000.0,0.000116,3.545277e-08
134,18324000.0,19072000.0,0.000031,1.720313e-08
135,19072000.0,19850000.0,0.000032,1.883995e-08


I is possible to extract some useful information about the code and the simulation settings: 

In [5]:
version = openmc_results.get_openmc_version
batches = openmc_results.get_batches
ppbatch = openmc_results.get_particles_per_batch

print(f'Code version: {version}')
print(f'Batches: {batches}')
print(f'Particles per batch: {ppbatch}')

Code version: (0, 14, 0)
Batches: 100
Particles per batch: 1.00e+09


In [6]:
ofb.to_hdf(df=df,
hdf_file='openmc-0-14-0_endfb80.h5', 
tally_name='nspectrum',
xs_library='ENDFB-8.0', 
xaxis_name='energy low [eV]',
path_to_folder='results_database', 
when='July 2024', 
where='Department of Energy, Politecnico di Torino', 
code_version='openmc-0.14.0',
batches='100', 
particles_per_batch='1.00e+09',
)

/home/davide/anaconda3/envs/openmc0.14/lib/python3.12/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'energy low [eV]'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/davide/anaconda3/envs/openmc0.14/lib/python3.12/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'energy high [eV]'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/davide/anaconda3/envs/openmc0.14/lib/python3.12/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'std. dev.'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use 

In [7]:
# _foil_volume = .1 * 1.8**2/4 * np.pi
# volumes_onaxis1 = np.concatenate((np.ones(5), np.ones(4)*2, np.ones(4)*3)) * _foil_volume

df = openmc_results.get_tally_dataframe(tally_name='vessel_photon_spectra_tally')
del df['surface']
del df['particle']
del df['nuclide']
del df['score']

df

,energy low [eV],energy high [eV],mean,std. dev.
0,300000.0,400000.0,0.009535,2.958113e-07
1,400000.0,500000.0,0.011082,3.139553e-07
2,500000.0,600000.0,0.016374,3.534549e-07
3,600000.0,700000.0,0.009909,2.711692e-07
4,700000.0,800000.0,0.010747,2.935552e-07
5,800000.0,900000.0,0.012222,3.541105e-07
6,900000.0,1000000.0,0.009671,2.840615e-07
7,1000000.0,1100000.0,0.009206,3.114587e-07
8,1100000.0,1200000.0,0.009537,3.029122e-07
9,1200000.0,1300000.0,0.009371,2.690401e-07


In [8]:
ofb.to_hdf(df=df,
hdf_file='openmc-0-14-0_endfb80.h5', 
tally_name='gspectrum',
xs_library='ENDFB-8.0', 
xaxis_name='energy low [eV]',
path_to_folder='results_database', 
when='July 2024', 
where='Department of Energy, Politecnico di Torino', 
code_version='openmc-0.14.0',
batches='100', 
particles_per_batch='1.00e+09',
)

/home/davide/anaconda3/envs/openmc0.14/lib/python3.12/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'energy low [eV]'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/davide/anaconda3/envs/openmc0.14/lib/python3.12/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'energy high [eV]'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/davide/anaconda3/envs/openmc0.14/lib/python3.12/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'std. dev.'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use 

## Store results in a database-compatible HDF file
It is possible to store the postprocessed tally in a HDF file compatible with the other files already present in the `\results_database\` folders of the benchmarks available in `\model\`. Of course some information need to be given:
- The name of the HDF file (either existing or to create) to store the results in
- The name of the tally (should not change from the name in the `statepoint` file or, at least, it has to be consistent with the name present in the other HDF files in the database)

Additional:
- Normalization for postprocessing (e.g. cell volume)
- Name of the nuclear data library used (strongly recommended)
- When the simulation has been run
- Where it has been run

Let's verify that we stored something in our file:

In [9]:
openmc_file = ofb.ResultsFromDatabase(filename='openmc-0-14-0_endfb80.h5', path='results_database')
# openmc_file.get_tally_dataframe(tally_name='rr_onaxis1_nb93')

In [10]:
openmc_file.list_tallies()

<KeysViewHDF5 ['gspectrum', 'nspectrum']>


In [11]:
openmc_file.get_tally_dataframe(tally_name='gspectrum')

,energy low [eV],energy high [eV],mean,std. dev.
0,300000.0,400000.0,0.009535,2.958113e-07
1,400000.0,500000.0,0.011082,3.139553e-07
2,500000.0,600000.0,0.016374,3.534549e-07
3,600000.0,700000.0,0.009909,2.711692e-07
4,700000.0,800000.0,0.010747,2.935552e-07
5,800000.0,900000.0,0.012222,3.541105e-07
6,900000.0,1000000.0,0.009671,2.840615e-07
7,1000000.0,1100000.0,0.009206,3.114587e-07
8,1100000.0,1200000.0,0.009537,3.029122e-07
9,1200000.0,1300000.0,0.009371,2.690401e-07


In [12]:
openmc_file.print_all_info()

Info:
 When: July 2024
 Where: Department of Energy, Politecnico di Torino
 Code version: openmc-0.14.0
 XS library: ENDFB-8.0
 Literature: n/a



# VERIFICA

In [5]:
openmc_endfb80_file = ofb.ResultsFromDatabase('openmc-0-14-0_endfb80.h5', path='results_database')

In [7]:
openmc_endfb80_file.get_tally_dataframe(tally_name='gspectrum')

,energy low [eV],energy high [eV],mean,std. dev.
0,300000.0,400000.0,0.009521,3.008957e-05
1,400000.0,500000.0,0.011138,3.208169e-05
2,500000.0,600000.0,0.016383,3.790297e-05
3,600000.0,700000.0,0.009903,3.361555e-05
4,700000.0,800000.0,0.010794,3.531472e-05
5,800000.0,900000.0,0.012235,3.272543e-05
6,900000.0,1000000.0,0.009624,3.290314e-05
7,1000000.0,1100000.0,0.009230,3.621177e-05
8,1100000.0,1200000.0,0.009506,3.159216e-05
9,1200000.0,1300000.0,0.009341,2.697091e-05


In [8]:
openmc_endfb80_file.print_all_info()

Info:
 When: 31/07/2024
 Where: Politecnico di Torino
 Code version: openmc-0.14.0
 XS library: endfb80
 Literature: n/a



In [9]:
openmc_endfb80_file.get_tally_dataframe(tally_name='nspectrum')

,energy low [eV],energy high [eV],mean,std. dev.
0,86140.0,89650.0,0.000864,1.111672e-07
1,89650.0,93310.0,0.000967,9.781908e-08
2,93310.0,97120.0,0.000967,1.123191e-07
3,97120.0,101090.0,0.000972,1.183270e-07
4,101090.0,105210.0,0.001070,1.169516e-07
...,...,...,...,...
132,16915000.0,17605000.0,0.000113,3.088201e-08
133,17605000.0,18324000.0,0.000116,3.545277e-08
134,18324000.0,19072000.0,0.000031,1.720313e-08
135,19072000.0,19850000.0,0.000032,1.883995e-08
